In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('./train.csv')
x_train = np.array(df.iloc[:,1:]) # array, each element being an array of 784 elements, need to be rearranged in a 28*28 array for the image
y_train = np.array(df.iloc[:,0]) # labels of the pictures
print(x_train.shape)
num_samples = x_train.shape[0]
X_train = x_train.reshape(num_samples,28,28)
# Normalising the dataset
X_train = X_train / 255



In [ ]:
print(np.unique(y_train))
#Checking the number of unique labels

In [ ]:
# Creating Validation data and training data
X_train = X_train.reshape(-1,28,28,1)
from sklearn.model_selection import train_test_split
# using this because we want the validation data to be randomly taken from the dataset, we don't want to have any kind of relation in the data, knowingly or unknowingly
inp_train, inp_validation, labels_train, labels_validation = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42  # 20% validation set
)
print(inp_train.shape, inp_validation.shape,labels_train.shape, labels_validation.shape)

In [ ]:
# Defining the architecture of the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def build_model(input_shape=(28, 28, 1), num_classes=25):  # num_classes give the total number of outputs
    model = Sequential()

    # Convolutional Block 1
    model.add(Conv2D(16, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    

    # Convolutional Block 2
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))


    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

# Example: Build and summarize the model
model = build_model()
model.summary()


In [ ]:
#Compiling the model using adam
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  # Use sparse if labels are integers, categorical if one-hot encoded
    metrics=['accuracy']
)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_loss',  # Monitor validation loss
    patience=1,          # Stop after 1 epochs without improvement
    restore_best_weights=True  # Restore the best weights
)

In [ ]:
history = model.fit(
    inp_train, labels_train,
    epochs=10,
    batch_size=32,
    validation_data=(inp_validation,labels_validation),
    callbacks = [early_stopping]
)

def plot_training_history(history):
    # Extract values from the history object
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    # Plot accuracy
    ax1.plot(acc, label='Training Accuracy')
    ax1.plot(val_acc, label='Validation Accuracy')
    ax1.set_title('Accuracy')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Accuracy')
    ax1.legend()

    # Plot loss
    ax2.plot(loss, label='Training Loss')
    ax2.plot(val_loss, label='Validation Loss')
    ax2.set_title('Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.legend()

    # Show the plot
    plt.tight_layout()
    plt.show()

# Call the function to plot the training history
plot_training_history(history)


In [ ]:
#Now preparing the testing data
df2 = pd.read_csv('./test.csv')

x_test = np.array(df2.iloc[:,1:]) # array, each element being an array of 784 elements, need to be rearranged in a 28*28 array for the image
y_test = np.array(df2.iloc[:,0]) # id of the pictures

num_samples = x_test.shape[0]
X_test = x_test.reshape(num_samples,28,28)

# Normalising the dataset
X_test = X_test / 255
X_test= X_test.reshape(-1,28,28,1)


In [ ]:
# Making the predictions
predictions = model.predict(X_test)

# Get the class with the highest probability for each image
predicted_classes = np.argmax(predictions, axis=1)



In [ ]:
#Writing the got predictions in a csv file
import pandas as pd

# Create a DataFrame with image indices and their predicted classes
prediction_df = pd.DataFrame({
    "id": range(len(predicted_classes)),
    "Predicted Class": predicted_classes
})

# Save the DataFrame to a CSV file
prediction_df.to_csv("this_is_new_prediction3.csv", index=False)
